# 1. Load modules

In [1]:
%matplotlib widget 
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import matplotlib.ticker as ticker
from scipy import optimize
import os, sys
from pathlib import Path
import xrdpy

In [ ]:
print(f'xrdpy version: {xrdpy.__version__}')
print(f'xrdpy path: {xrdpy.__path__}')

# 2. Set up the xrd-file path

In [2]:
xrdpy_test_path = Path().absolute()

In [3]:
filepath = os.path.join(xrdpy_test_path,"xrd_files")
filename = "NT3334C_RSM_w2T-2T-105AlN-AlGaN_fast.xrdml"

In [4]:
savefig = True
save_fig_path = os.path.join(xrdpy_test_path,"../imgs")

In [5]:
gen_fns = xrdpy.general_fns(print_log='low')
xrd_ = xrdpy.xrd(print_log='low')
xrd_plt = xrdpy.plottings(save_figure_dir=save_fig_path, print_log='low')

# 3. Set default variables

#### Variable source: https://www.ioffe.ru/SVA/NSM/Semicond/

In [6]:
alloy_type='ternary' 
structure_type='wz'

# Lattice parameters for AlN and GaN binaries
AlN_a, AlN_c = 3.112, 4.982
GaN_a, GaN_c = 3.189, 5.185    

# Cij parameters for AlN and GaN binaries
GaN_C13 = 106
GaN_C33 = 398
AlN_C13 = 108
AlN_C33 = 373

# Bowing parameters
bowing_a = 0.0
bowing_c = 0.0
bowing_C13 = 0.0
bowing_C33 = 0.0

# reciprocal axes are multiplited by this number to make enlarge axes
mul_fact = [10000, 10000] # along [Qx, Qy]

# a_lp_bin: [a_lattice_parameter bin_1, a_lattice_parameter bin_2, bowing]
# For ternary_wz we need followings: [a_lp_bin, c_lp_bin, c13_bin, c33_bin]
# Alloy_params = x*list_binary_parameters[i][0] + (1-x)*list_binary_parameters[i][1]+bowing*x*1-x
# WARNING: Maintain the order of data given for material parameters for all list below. 
# E.g. Do not do like [[AlN_data, GaN_data, bowing],[GaN_data, AlN_data, bowing],...]
list_binary_parameters = [[AlN_a, GaN_a, bowing_a], [AlN_c, GaN_c, bowing_c],
                          [AlN_C13, GaN_C13, bowing_C13], [AlN_C33, GaN_C33, bowing_C33]]

# 4. Read xrd-file

## 4.1 Real space data

In [7]:
lambda_wavelength, two_theta_values, omega_values, rsm_intesity  = \
    xrd_.xrd_read_data(xrd_file_name=os.path.join(filepath, filename))
# X-ray wave number
R = 1/lambda_wavelength

Wavelength used (from xrd file): 1.5405980
Intensity unit (from xrd file): counts/seconds


## 4.2 Reciprocal space conversion

In [8]:
total_two_theta_in_row = np.shape(rsm_intesity)[1]
rec_space_x, rec_space_y = xrd_.Qxy(omega=omega_values, two_theta=two_theta_values, 
                                    total_two_theta_in_row=total_two_theta_in_row,
                                    R=R, mul_fact=mul_fact)

# 5. Plottings

## 5.1 Real space map

In [9]:
print('Plotting real space map ...')
xlabel_text = r'2$\mathrm{\theta}$'
ylabel_text = r'$\omega$ / $2\theta$'
save_file_name = 'AlN_AlGaN_AlN_real_space.png' if savefig else None
_ = xrd_.xrd_plot(save_figure_dir=save_fig_path, save_file_name=save_file_name, 
                  mode="real_space", xaxis_label=xlabel_text,threshold_intensity=1e-5,
                  yaxis_label=ylabel_text, color_map='jet', color_scale='log', 
                  colorbar_label='Intensity (counts/s)', show_plot=True, 
                  savefig=savefig, dpi=75)
print('Done')

Plotting real space map ...
Done


## 5.2 Reciprocal space map (without shift)

### 5.2.1 Reciprocal space map (single)

In [10]:
xlabel_text = f'Qx*{mul_fact[0]} (rlu)'
ylabel_text = f'Qy*{mul_fact[1]} (rlu)'
print('Plotting reciprocal space map (without any shift)...')
save_file_name = 'AlN_AlGaN_AlN_reciprocal_space.png' if savefig else None
_ = xrd_plt.xrd_plot(save_file_name=save_file_name, x_values=rec_space_x, 
                     y_values=rec_space_y, z_values=rsm_intesity,
                     mode="reciprocal_space", xaxis_label=xlabel_text, threshold_intensity=0.1,
                     yaxis_label=ylabel_text, color_map='jet', color_scale='log',
                     colorbar_label='Intensity (counts/s)', savefig=savefig, dpi=75)
print('Done')

Plotting reciprocal space map (without any shift)...
Done
